# Milestone 2

In [ ]:
from cProfile import label

from altair import XOffset
from matplotlib.pyplot import ylabel, title
from networkx.algorithms.bipartite.basic import color
from scipy.stats import binned_statistic
from streamlit import columns
%load_ext autoreload
%autoreload 2

In [ ]:
# all imports
from src import *
import pandas as pd
import os.path

## Data acquisition

In [ ]:
filename = './data/dataframe_2016_to_2019.csv'
start_year = 2016
end_year = 2019

if not os.path.isfile(filename):
    # Get the data from the NHL API (2016 - 2019)
    nhl_data_provider = get_data_from(start_year, end_year)

    # Clean the data
    clean_regular_season, clean_playoff = clean_data(nhl_data_provider)

    # Transform data into dataframe
    df_2016_to_2019 = convert_dictionaries_to_dataframes(clean_regular_season,
                                                         clean_playoff,
                                                         np.arange(start_year, end_year + 1).tolist())
    df_2016_to_2019.to_csv(filename, index=False)

In [ ]:
train_data = pd.read_csv(filename)

# Ingénierie des caractéristiques

## Question 1

In [ ]:
train_data['isGoal'] = train_data['typeDescKey'].apply(lambda x: 1 if x == 'goal' else 0)
train_data['emptyGoalNet'] = train_data['emptyGoalNet'].astype(int)

In [ ]:
count_table = pd.crosstab(train_data['isGoal'], train_data['typeDescKey'])
# count_table2 = pd.crosstab(train_data['emptyGoalNet_0_1'],train_data['emptyGoalNet'])

count_table
# count_table2

#### Histogramme du nombre de tirs (buts et non-buts séparés), regroupés par distance

In [ ]:
## Add in advancedVisualisation (by Youry)
def histogram_2_variables(df, x, hue, bins, xlabel, ylabel, title, legendTitle, legendLabels):
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=x, hue=hue, bins=bins, multiple='dodge')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend(title=legendTitle, labels=legendLabels)
    plt.show()


histogram_2_variables(df=train_data, x='shotDistance', hue='isGoal', bins=20, xlabel='Distance du tir',
                      ylabel='Nombre de tirs', title='Histogramme du nombre de tirs par distance (buts et non-buts)',
                      legendTitle='Est-ce un but?', legendLabels=['But', 'Non-but'])

#### Histogramme du nombre de tirs (buts et non-buts séparés), regroupés par angle

In [ ]:
histogram_2_variables(train_data, x='shotAngle', hue='isGoal', bins=20, xlabel='Angle du tir', ylabel='Nombre de tirs',
                      title='Histogramme du nombre de tirs par angle (buts et non-buts)', legendTitle='Est-ce un but?',
                      legendLabels=['But', 'Non-but'])

#### Histogramme 2D où un axe est la distance et l'autre est l'angle (sans distinction entre buts et non-buts)

In [ ]:
def histogram2D_2_variables(df, x, y, bins, label, xlabel, ylabel, title):
    # Remove rows with NaN values
    df = df.dropna(subset=[x, y])
    # Plot the histogram with hue for 'is_goal'
    plt.figure(figsize=(10, 6))
    plt.hist2d(df[x], df[y], bins=bins, cmap='Blues')
    # Customize legend labels for hue values
    plt.colorbar(label=label)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()


histogram2D_2_variables(train_data, x='shotDistance', y='shotAngle', bins=20, label='Nombre de tirs',
                        xlabel='Distance du tir', ylabel='Angle du tir',
                        title='Histogramme 2D du nombre de tirs par distance et angle')

In [ ]:
def histogram2D_2_variables_seaborn(df, x, y, kind, title, xlabel, ylabel):
    # Remove rows with NaN values
    df = df.dropna(subset=[x, y])
    # Jointplot to visualize the 2D histogram
    sns.jointplot(data=df, x=x, y=y, kind=kind, cmap='Blues')
    # Add title and labels
    plt.suptitle(title, fontsize=16)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    # Show the plot
    plt.show()


histogram2D_2_variables_seaborn(train_data, x='shotDistance', y='shotAngle', kind='hist',
                                title="Histogramme 2D de la Distance et de l'Angle", xlabel='Distance du tir',
                                ylabel='Angle du tir')

## Question 2

#### Le taux de but (buts / (buts + non-buts)) par distance

In [ ]:
df_distance_goal_rate = train_data.groupby('shotDistance')['isGoal'].mean().reset_index()
df_distance_goal_rate.columns = ['shotDistance', 'goalRate']

df_distance_goal_rate


In [ ]:
def lineplot_2_variables(df, x, y, title, xlabel, ylabel):
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x=x, y=y, marker='o')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.show()


lineplot_2_variables(df=df_distance_goal_rate, x='shotDistance', y='goalRate',
                     title='Taux de but en fonction de la distance', xlabel='Distance du tir', ylabel='Taux de but')

In [ ]:
def scatterplot_2_variables(df, x, y, color, s, title, xlabel, ylabel):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x=x, y=y, color=color, s=s, marker='o')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.show()


scatterplot_2_variables(df=df_distance_goal_rate, x='shotDistance', y='goalRate', color='green', s=20,
                        title='Taux de but en fonction de la distance', xlabel='Distance du tir', ylabel='Taux de but')

#### Le taux de but (buts / (buts + non-buts)) par angle

In [ ]:
df_angle_goal_rate = train_data.groupby('shotAngle')['isGoal'].mean().reset_index()
df_angle_goal_rate.columns = ['shotAngle', 'goalRate']

df_angle_goal_rate


In [ ]:
scatterplot_2_variables(df=df_angle_goal_rate, x='shotAngle', y='goalRate', color='green', s=20,
                        title='Taux de but en fonction de l\'angle', xlabel='Angle du tir', ylabel='Taux de but')

## Question 3

#### Verifier but net non vide sur l'équipe adverse depuis l'intérieur de votre zone défensive

In [ ]:
plot_goals_by_distance(train_data)

In [ ]:
faraway_goals = train_data[(train_data['shotDistance'] > 30) & (train_data['typeDescKey'] == 'goal')]

grouped_df = faraway_goals.groupby(['shotType']).count()

print(grouped_df['idGame'])

In [ ]:
faraway_goals[faraway_goals['shotType'] == 'wrap-around']

In [ ]:
# pour l'evenement 47255 du match 2016020779, le type de tir n'est pas correct car un wrap-around ne peut pas se faire aussi loin
# voir: https://www.nhl.com/gamecenter/pit-vs-stl/2017/02/04/2016020779/playbyplay
# pour l'evenement 135239 du match 2017020853, le type de tir n'est pas correct car un wrap-around ne peut pas se faire aussi loin
# voir: https://www.nhl.com/gamecenter/pit-vs-stl/2018/02/11/2017020853/playbyplay

# video wrap-around: https://www.youtube.com/watch?v=tmRibUXW8RE

In [ ]:
defensive_zone_goals = train_data[(train_data['shotDistance'] > 80) & (train_data['typeDescKey'] == 'goal')]

grouped_df = defensive_zone_goals.groupby(['shotType', 'emptyGoalNet']).count()

print(grouped_df['idGame'])

In [ ]:
defensive_zone_goals[(defensive_zone_goals['shotType'] == 'backhand') & (defensive_zone_goals['emptyGoalNet'] == False)]

In [ ]:
# l'venement 54362 du match 2016020894 n'a pas les bonnes coordonées. 
# voir la vidéo: https://www.youtube.com/watch?v=lM6JXVW0-YY du match. le but de Mats Zuccarello en période de shoot-out
# est fait du coté droit, non du coté gauche

# l'evenemnt 80622 du match 2017020004 n'a pas les bonnes coordonées.
# voir la vidéo: https://www.youtube.com/watch?v=WpIGr7srlLY. le but de Kevin Labanc est du mauvais coté

In [ ]:
defensive_zone_goals[(defensive_zone_goals['emptyGoalNet'] == False) & (defensive_zone_goals['shotDistance'] > 110)]

In [ ]:
# l'evenemnet 1806 n,est pas le bon zoneShoot/zoneCode. il a comme zoneShoot D (defensive) alors que quand on regarde le but de
# John Tavares, il le marque bien dans la zone offensive https://www.youtube.com/watch?v=MO7vAygX2_c

# Modèles de base

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#### Préparation de la caractéristique et de la cible

In [ ]:
# Préparation de la caractéristique et de la cible
X = train_data[['shotDistance']].dropna()
y = train_data['isGoal'][X.index]

In [ ]:
# Division des données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [ ]:
# # Vérification des types et absence de NaN
# print("Type de X_train:", type(X_train))
# print("Type de y_train:", type(y_train))
# print("Valeurs manquantes dans X_train:", X_train.isnull().sum().sum())
# print("Valeurs manquantes dans y_train:", y_train.isnull().sum())

#### Entraîner le modèle

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

#### Prédiction et évaluation

In [ ]:
# Prédiction et évaluation
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Précision du modèle sur l'ensemble de validation: {accuracy:.2f}")

## Question 5

### Ajout de l'artefact sur Wandb

In [ ]:
import wandb

# Connect to the Wandb API
api = wandb.Api()

# Spécifiez votre projet
project_name = "IFT6758.2024-A11" 
entity = "youry-macius-universite-de-montreal"  

try:
    artifact = api.artifact(f'{entity}/{project_name}/wpg_v_wsh_2017021065:latest')
    print("L'artefact existe déjà.")
except wandb.errors.CommError as e:
    run = wandb.init(project="IFT6758.2024-A11")
    
    # Create artefact
    artifact = wandb.Artifact(
        "wpg_v_wsh_2017021065",
        type="dataset"
    )
    
    # add data
    my_table = wandb.Table(dataframe=train_data[train_data['idGame'] == 2017021065])
    artifact.add(my_table, "wpg_v_wsh_2017021065")
    run.log_artifact(artifact)
